In [ ]:
import project_lib as mylib
import numpy as np
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
cores = mp.cpu_count()
print 'cores: %s'%cores
CORES = [cores if cores<20 else 20 for i in range(1)][0] # number of cores I intend to attempt to use
%matplotlib notebook

In [ ]:
###############################################################################
# global input parameters needed for this run
log_qub = 4
schedname='linear'
#define number of logical and ancilla qubits
anc_qub = log_qub
qubits = log_qub*2
Qubits = qubits

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h_i = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J_ij = np.concatenate((Jt,Jb),axis=0) # the full J

#now lets run it with the intended annealing schedule
theta = 71.1#degrees

In [ ]:
sfl = lambda points: np.cos(theta*np.pi/180.)*mylib.linear_schedule(points)
sfa = lambda points: np.sin(theta*np.pi/180.)*mylib.linear_schedule(points)
sfB = mylib.linear_schedule
sched_funcs = [[sfl]*log_qub+[sfa]*log_qub,[sfB]]
a = mylib.Anneal(Qubits,[h_i,J_ij],T=100.,points = 1000,sched_funcs = sched_funcs, diff_scheds = True, show_bar = True, light = False) 
a.run()
a.show_results()

Now that we have this anneal object I want to find the probability of the system being in each of the ground states throughout the anneal as well as the total probability of being on one of them. I will need to mess around with some of my library code for this

In [ ]:
# import dependencies
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
from project_lib.progressbar import printProgress
import os.path
import scipy.linalg # this line is needed as well as the one below so that this imports in Jupyter correctly
import scipy as sp
from tabulate import tabulate
import time
import sys
print 'annealing loaded - '+'%s/%s/%s - '%time.localtime()[:3][::-1]+':'.join(('0'+str(x))[-2:] for x in list(time.localtime()[3:6]))
########################################### SIGMA MATRICES ###########################################################
# define the pauli matrices we are using
sigma_x = np.matrix('0. 1.; 1. 0.')
sigma_z = np.matrix('1. 0.; 0. -1.')

def sigma_zi(i,N):
    '''This returns the equivalent sigma_z for the ith qubit in the new Hilbert space given that there are N qubits'''
    return np.kron(np.eye(2**i), np.kron(sigma_z,np.eye(2**(N-(i+1)))))
    
def sigma_xi(i,N):
    '''This returns the equivalent sigma_x for the ith qubit in the new Hilbert space given that there are N qubits'''
    return np.kron(np.eye(2**i), np.kron(sigma_x,np.eye(2**(N-(i+1)))))
    
    
########################################### SAVE AND LOAD ############################################################
def save_project_data(filename,data):
    path = r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Data"
    completeName = os.path.join(path,filename)
    np.savetxt(completeName,data)

def load_project_data(filename):
    path = r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Data"
    completeName = os.path.join(path,filename)
    return np.loadtxt(completeName)



####################################### DIAGONALISATION FUNCTIONS ####################################################
# the Anneal class is dependent on the next two functions
def Diagonaliser(M):
    '''This function takes a hermitian matrix and return the unitary transform matrices to change to and from the eigenbasis
    as well as the diagonalised form of this matrix'''
    eigvals, eigvecs = sp.linalg.eigh(M)
    P = np.matrix(eigvecs)
    P_in = P.H
    return eigvals,P,P_in
    
def make_diag(d):
    '''this is a function to make a diagonal matrix given a series of values to go on the diagonal'''
    return np.matrix(np.diag(d))
    


####################################### ANNEALING SCHEDULE FUNCTIONS #################################################
# use the following function as the default anealing schedule
def linear_schedule(points):
    '''A(s) is indexed with return[0] B(s) is indexed with return[1]'''
    return np.asarray([np.ones(points) - np.linspace(0,1,points),np.linspace(0,1,points)])


#################################### BACK AND FORTH TO BIT SOLUTIONS #################################################
def bit_values(qubits,v):
    '''this function takes the vector in the hamiltonian space and turns it into an expectation bit value'''
    bits = np.zeros(qubits)
    for i in range(qubits):
        H = sigma_zi(i,qubits)
        bits[i] = 0.5*(1-v.getH()*H*v)
    return bits

def make_eigen_vector(bits):
    '''this function takes a string of bit assignments and turns them into a vector in the hamiltonian space'''
    neg = np.matrix('0. 1.')
    pos = np.matrix('1. 0.')
    vecs = [pos,neg]
    vec = vecs[bits[0]]
    for i in range(1,len(bits)):
        vec = np.kron(vec,vecs[bits[i]])
    return vec

def bit_table(last_state,problem_x0s, qubits):
    '''given a state this fucntion prints out the table of the probability of measuring this state in it's eigenfunctions'''
    headers = ['Bit Solution','Final State Probability','Problem X0 Probability']
    bits = []
    last_state_probs = []
    problem_x0_probs = []
    table = []
    for i in range(2**qubits):
        bits.append( [(i/2**j)%2 for j in range(qubits)[::-1]])
        last_state_probs.append((abs(last_state.getH()*np.transpose(make_eigen_vector(bits[-1])))**2).item(0))
        problem_x0_probs.append(sum([abs(np.dot(make_eigen_vector(bits[-1]),prob_x0_pos)**2).item(0) for prob_x0_pos in problem_x0s]))
        if problem_x0_probs[-1]==0: table.append([bits[-1],last_state_probs[-1],''])
        else: table.append([bits[-1],last_state_probs[-1],problem_x0_probs[-1]])
    
    table.insert(0,['','Result Highlights',''])
    table.insert(1,['','Full Results',''])
    nonzeroindex = np.where(np.asarray(problem_x0_probs) !=0)[0]
    for nzi in nonzeroindex:
        table.insert(1,[bits[nzi],last_state_probs[nzi],problem_x0_probs[nzi]])
    table.append(['Total_Prob - 1: ',float(np.sum(last_state_probs)-1),''])
    print tabulate(table, headers, tablefmt="grid")
    return tabulate(table, headers, tablefmt="grid")
    
def Transfer(Psi,psi0s, d,P,P_in, k,dt):
    '''
    This function returns all of the transfer rates out of the ground state to the other states up to the kth one
    '''
    Tr_k = []
    U = P*make_diag(np.exp(-1.j*d*dt))*P_in
    p0s = [np.kron(psi0.H,psi0) for psi0 in psi0s] # the projection operators onto the ground state
    for ki in range(k):
        psi_ki = P[:,np.where(d==min(d[d!=min(d)]))]
        project_ki = np.kron(psi_ki.H,psi_ki)
        Tr_k.append(sum([sum(abs(project_ki*U*p0*Psi)**2).item(0) for p0 in p0s]))
    return np.asarray(Tr_k)
    

class Anneal():
    '''
    This object is to keep all the annealing stuff together in one neat unit for each anneal completed.
    It applies an anneal for any number of bits as long as you give it the right input. 
    
    Each function explains itself but generally: 
        -the run function does the anneal
        -show_results shows the resuls of the anneal
    
    You need to input the parameters in the form:
        params = [h,j]
        
    Furthermore J needs to be formatted correctly. If there are n bits and Jij is the interaction coefficient between the ith and jth bit,
    J can be in the following format: 
        J = [ [J12,J13,J14,...Jin],  [J23,J24,J25,...J2n],  .......,  [J(n-2)(n-1),J(n-2)n],  [J(n-1)n] ]
        Alternatively it can be in a square array where J[i,j] = Jij and all terms below the upper right triange are zeros (this includes the diagonal)
    '''
    
    def __init__(self,qubits,params, **kwargs):
        '''This function sets up the attributes of the class'''
        # management
        self.creation_date = time.gmtime()
        self.light = kwargs.get('light', False) # runs a version of this where we are only interested in the final success probability nothing else at all
        if type(self.light)==str: # should be 'DEG=N'
            self.DEG = int(self.light[4:])
            self.light=True
        self.PITNUM = kwargs.get('PROGRESS_IT_NUMBER', None)
        self.show_bar = kwargs.get('show_bar', True)
        self.k = kwargs.get('k_states', 9)
        
        # schedule functions
        self.diff_scheds = kwargs.get('diff_scheds', False) # I am assuming the form of sched_funcs will be [ [A1(s), A2(s),...,AQUBITS(S)] , [B(S)] ]
        if self.diff_scheds: schedules = kwargs.get('sched_funcs') 
        else: schedule = kwargs.get('sched_func', linear_schedule)        
        
        # parameters
        self.qubits = qubits
        self.points = kwargs.get('points', 10000)
        self.T = float(kwargs.get('T', 100))
        self.h = params[0]
        self.J = params[1]
        if type(params[1])==list: self.J = np.asarray([[0]*(i+1)+params[1][i] for i in range(qubits-1)]+[[0]*qubits])
        self.check_params() # checks validity
        self.hamiltonian_parts() # sets up Hp and Hb
        
        # set schedules for the anneal
        self.ss = np.linspace(0,1,self.points)
        if not self.diff_scheds:
                self.AB = schedule(self.points)
        else: 
            ABs = np.zeros((self.points,qubits+1))
            for i in range(qubits):
                ABs[:,i] = schedules[0][i](self.points)[0]
            ABs[:,-1] = schedules[1][0](self.points)[1]
            self.ABs = ABs
        
        
    
    def check_params(self):
      '''Checks to see of the parameters you have put in are consistant with the number of qubits you have put in'''
      cond1 = (self.J.shape==(self.qubits,self.qubits))
      cond2 = (len(self.h)==self.qubits)
      if not cond1: 
          print '\n\n\nThe length of the J parameters array does not match the number of qubits you have selected\n\n\n'
      if not cond2: 
          print '\n\n\nThe length of the h parameters array does not match the number of qubits you have selected\n\n\n'    
        
    def hamiltonian_parts(self):
        '''
        - Constructs the hamiltonian from the parameters input
        - If we are doing an anneal run where we have different annealing schedules for each qubits it will return a list 
        of hamiltonian parts for the base hamiltonian. Else it returns a single hamiltonian for this part
        '''
        #initiate the hamiltonians giving them the right dimensions
        Hb = []
        Hp = np.matrix(np.eye(2**self.qubits)*0.)
        h = self.h
        J = self.J
        
        
        # loop through adding pieces to construct the hamiltonians 
        for i in range(self.qubits):
            Hb.append(sigma_xi(i,self.qubits))
            Hp += h[i]*sigma_zi(i,self.qubits)
            # the following part adds the interaction between all qubits. There is 1 interaction coefficient for each pairing 
            for j in range(i+1,self.qubits): 
                Hp +=J[i][j]*sigma_zi(i,self.qubits)*sigma_zi(j,self.qubits)     
        if not self.diff_scheds: Hb = sum(Hb)
        #set attibutes to store data   
        self.Hb = Hb
        self.Hp = Hp
    
    def H(self,i):
        if not self.diff_scheds:
            Hi = -self.AB[0][i]*self.Hb + self.AB[1][i]*self.Hp
        else: 
            Hi = sum([-self.ABs[:,j][i]*self.Hb[j] for j in range(self.qubits)])+self.ABs[:,-1][i]*self.Hp
        return Hi
    

    def euler_update(self,dt,i):
        # mathematical pieces to diagonalise
        d,P,P_in = Diagonaliser(self.H(i))
        # store energy eigenvalues
        self.eigenvals.append(d)
        # store the new state of the qubits
        self.states.append(P*make_diag(np.exp(-1.j*d*dt))*P_in*self.states[-1])
        # the following is not needed in light mode
        if not self.light:
            # find instantaneous ground states
            self.instant_x0s.append([P[:,d==min(d)][:,k] for k in range(P[:,d==min(d)].shape[1])]) #  this line can't come before the line above or things will change due to the apend. This is the current state in the diagonal basis
            # find and store the probability of measuring the qubits to be in the instantaneous ground state
            self.instant_x0s_prob.append(sum([abs(np.dot(self.states[-1].getH(),ins_x0_pos)**2).item(0) for ins_x0_pos in self.instant_x0s[-1] ])) # this line finds the probability of the state being in the instantaneous ground state
            # find and store the probability of measuring the qubits to be in the problem ground state
            self.problem_x0_prob.append(sum([abs(np.dot(self.states[-1].getH(),prob_x0_pos)**2).item(0) for prob_x0_pos in self.problem_x0s]))
            # find and store the energy eigenvalue gap
            self.delta_eigenvals.append(min(d[d!=min(d)])-min(d))
            for k_i in range(self.k):
                self.prob_in_kth_state[k_i][i]=abs(np.dot(self.states[-1].getH(),P[:,k_i])**2).item(0)
            if i>0:
                self.Tr_i[:,i] = Transfer(self.states[-2],self.instant_x0s[-2], d,P,P_in, self.k,dt)
        
    def run(self):
        '''this function runs the annealing protocol and creates attributes to store the data within the class'''
        
        # find the ground state of the base hamiltonian
        base_eigen_vals0,base_eigen_vecs0 = np.linalg.eigh(self.H(0)) # the minus sign before the base hamiltonian is because we use H(s) = -A(s)Hb + B(s)Hp and H(0) = -Hb
        self.base_x0 = base_eigen_vecs0[:,base_eigen_vals0.argmin()]+0.j*base_eigen_vecs0[:,base_eigen_vals0.argmin()]
        
        # we need to distinguish between the ground state of the base hamiltonian and the base state of the hamiltonian at t=0 because these may be different
        start_eigen_vals0,start_eigen_vecs0 = np.linalg.eigh(self.H(0))
        self.start_x0 =start_eigen_vecs0[:,start_eigen_vals0.argmin()]+0.j*start_eigen_vecs0[:,start_eigen_vals0.argmin()]
        
        # find the ground state of the problem hamiltonian 
        d,P = np.linalg.eigh(self.Hp)
        self.problem_x0s = [P[:,d==min(d)][:,i] for i in range(P[:,d==min(d)].shape[1])]       
        
        # set the time steps
        dt = self.T/self.points
        
        # set up data stores for the anneal whilst setting the inital state to be th ground state on the base hamiltonian
        self.states = [self.base_x0] # store the vector wavestates
        self.instant_x0s = [] # store the instantaneous ground state

        self.eigenvals =[] # store the energy eigenvalues
        self.delta_eigenvals = []
        
        self.problem_x0_prob = [] # store the probabilities of being measured in the problem base state
        self.instant_x0s_prob = [] # store the probabilities of being measured in the instantaneous base state        
        self.prob_in_kth_state=np.zeros((self.k,self.points))
        self.Tr_i = np.zeros((self.k,self.points))
        
        # carry out the anneal whilst storing the state and the energy eigenvalues thoughout the process
        l = len(self.ss)-1
        
  
        for i in range(len(self.ss)):
            # print loading bar but only update after each full percent has been completed
            if (((i+1)*100%(l+1)<=100) or i == l) and self.show_bar: printProgress (i, l, prefix = 'Annealing: ', suffix = 'est_time', decimals = 0, barLength = 50, PROGRESS_IT_NUMBER = self.PITNUM)
            self.euler_update(dt,i)
            if self.light and i == self.points-1: # calculate only the last probability of success
                self.problem_x0_prob.append(sum([abs(np.dot(self.states[-1].getH(),prob_x0_pos)**2).item(0) for prob_x0_pos in self.problem_x0s])) 

       
        # set attributes to store the data
        self.states = self.states[1:]
        # self.instant_x0s = instant_x0s I have decided I don't need these stored

        self.eigenvals = np.asarray(self.eigenvals)
        self.delta_eigenvals = np.asarray(self.delta_eigenvals)
        
        self.problem_x0_prob = np.asarray(self.problem_x0_prob)
        self.instant_x0s_prob = np.asarray(self.instant_x0s_prob)
        
        
        del self.PITNUM # Don't need this anymore. It was just to see when running
        del self.show_bar
        
    def show_results(self):
        '''This function outputs all of the important data, either printed or graphed'''
        
        # plot all of the energy eigenvalue traces
        print 'last set of eigenvalues were:',self.eigenvals[-1]
        plt.figure()
        plt.xlabel('s')
        plt.ylabel('$\lambda$')        
        for i in range(len(self.eigenvals[0,:])):
            plt.plot(self.ss,self.eigenvals[:,i])
        plt.show()
        
        # plot the probability of being found in the problem ground state/instanteaous gorund state
        plt.figure()
        ax1 = plt.subplot2grid((3,1), (0, 0), rowspan=2); ax2 = ax1.twinx()
        ax1.set_ylabel('$P(Problem\,ground\,state)$', color='r') 
        ax1.plot(self.ss, self.problem_x0_prob, 'r-')
        ax2.set_ylabel('$P(Instantaneous\,ground\,state)$', color='b')       
        ax2.plot(self.ss, self.instant_x0s_prob, 'b-')
        
        # plot the energy gap between the first 2 levels and the annealing schedule if it is uniform across bits
        ax3 = plt.subplot2grid((3,1), (2, 0), rowspan = 1,sharex = ax1)
        if not self.diff_scheds : 
            ax4 = ax3.twinx()
            ax4.set_ylabel('A(s), B(s)')
            ax4.plot(self.ss,self.AB[0], label = '$A(s)$',color = 'red')
            ax4.plot(self.ss,self.AB[1], label = '$B(s)$', color = 'blue')
        ax3.set_ylabel('$\Delta \lambda$',color = 'black')
        ax3.set_xlabel('s')     
        ax3.plot(self.ss,self.delta_eigenvals,color='black',label = '$\Delta \lambda$')    
        plt.legend(framealpha = 0, loc = 'best')
        plt.show()
        
        # if there has been different schedules we will plot these schedules seperately
        if self.diff_scheds:
            ax5 = plt.subplots()[1]
            ax5.set_ylabel('A(s), B(s)')
            for i in range(self.qubits):
                ax5.plot(self.ss,self.ABs[:,i], label = str(i+1),color = 'red', alpha = 0.5)
            ax5.plot(self.ss,self.ABs[:,-1], label = '$B(s)$', color = 'blue')
        plt.show()
        
        # print the bit table and the final probability of being in the ground state
        bit_table(self.states[-1],self.problem_x0s, self.qubits)
        print '\nProbability of being measured in lowest energy state: %s' %self.problem_x0_prob[-1]


In [ ]:
k=9
points=1000
k_i=0; step=2
np.zeros((k,points))[k_i][step]

In [ ]:
sfl = lambda points: np.cos(theta*np.pi/180.)*mylib.linear_schedule(points)
sfa = lambda points: np.sin(theta*np.pi/180.)*mylib.linear_schedule(points)
sfB = mylib.linear_schedule
sched_funcs = [[sfl]*log_qub+[sfa]*log_qub,[sfB]]
a = Anneal(Qubits,[h_i,J_ij],T=100.,points = 1000,k_states=20,sched_funcs = sched_funcs, diff_scheds = True, show_bar = True, light = False) 
a.run()
a.show_results()

In [ ]:
cumTr = np.cumsum(a.Tr_i,axis = 1)
max(cumTr[0])

In [ ]:
n0=2 # how many distinct levels
FS=20
s=np.linspace(0,1,a.points)
plt.figure(figsize = (13,6))
####################### TOP PLOT ############################
ax1=plt.subplot2grid((4,5),(0,0),colspan = 4,rowspan=2)
for i in range(n0+1):
    if i ==n0:
        plt.plot(s,sum(a.prob_in_kth_state[n0:8]),label='%s - %s'%(n0,7))    
    else:
        plt.plot(s,a.prob_in_kth_state[i],label=str(i))
plt.fill_between(s, np.zeros(s.size), sum(a.prob_in_kth_state[:8]), facecolor='black', alpha=0.2,)
plt.ylabel('$P_{k}(s)$',fontsize=FS)
plt.xlabel('$s$',fontsize=FS)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title = '$k$ th State')

####################### MIDDLE PLOT ############################
plt.subplot2grid((4,5),(2,0),colspan = 4,rowspan=1,sharex=ax1)
plt.plot(s,a.prob_in_kth_state[7],label=str(8))
plt.ylabel('$P_{k}(s)$',fontsize=FS)
plt.xlabel('$s$',fontsize=FS)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title = '$k$ th State')
plt.show()


####################### BOTTOM PLOT ############################
# we are going to make a cut off and only show the 4 excited states that have the highest average population
n1=3
Ex_AvPop = np.sum(a.prob_in_kth_state[8:],axis=1)
ind = np.argpartition(Ex_AvPop, -n1)[-n1:]

plt.subplot2grid((4,5),(3,0),colspan = 4,rowspan=1,sharex=ax1)
for i in ind:
    plt.plot(s,a.prob_in_kth_state[8+i],label=str(8+i))
#plt.fill_between(s, np.zeros(s.size), -sum(a.prob_in_kth_state[:8])+1, facecolor='black', alpha=0.2,)
plt.ylabel('$P_{k}(s)$',fontsize=FS)
plt.xlabel('$s$',fontsize=FS)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title = '$k$ th State')
plt.show()

In [ ]:
np.zeros((5,10))[:,0]